# Using Metropolis-Hastings MCMC to decode hidden text
Given scrambled passages from Tolstoy's *War and Peace*, I used the algorithm outlined in [Diaconis' paper](https://math.uchicago.edu/~shmuel/Network-course-readings/MCMCRev.pdf) to break the substitution cipher.  The first step was to create a model for true English language.  I didn't want to use *War and Peace* as corpus because in practice, I wouldn't know from where the ciphertext came.  I used a free sample of the [Wikipedia Corpus](https://www.english-corpora.org/wiki/), which has about 1.7 million words.  I had to do some preprocessing, so I wrote a simple python script *clean_text.py*.
  

These are some libraries that you might have to install:

In [16]:
# uncomment if you need to download these libraries
# !pip install word2number
# !pip install bs4
# !pip install unidecode
# !pip install pickle
# !pip install numpy

Import all the libraries that I need, as well as some helper scripts I wrote for preprocessing

In [17]:
from bs4 import BeautifulSoup
from word2number import w2n
import re
import string
import unidecode

import os.path

from collections import Counter
from itertools import product
import random

import pickle
import numpy as np
from collections import Counter

from clean_text import clean_text
# to expand contractions, I made this dictionary using Wikipedia's list of contractions
from contractions import contractions

Now let's clean the corpus

In [19]:
corpus = 'wiki.txt'

#handling this corpus (cleaning, etc)
if os.path.exists('cleaned_corpus.txt'):
	with open('cleaned_corpus.txt','r') as f:
		cleaned_corpus = f.read()
else:
	with open(corpus) as f:
		corpus = f.read()
		#nlp preprocessing
		cleaned_corpus = clean_text(corpus)
	with open('cleaned_corpus.txt','w') as f:
		f.write(cleaned_corpus)

corpus_sz = len(cleaned_corpus)

Some parameters I will reference throughout the script:

In [20]:
# A = {a,b,c,...,z,' '}
alphabet_string = string.ascii_lowercase+' '
alphabet = list(alphabet_string)
alphabet_sz = len(alphabet)

#machine eps, I got this neat trick from stack overflow
min_value = (7./3) - (4./3) -1
# n as in ngram
n = 5
all_possible_ngrams = product(alphabet,repeat = n)
# for consistency 
random.seed(13)
# param for accept probability
beta = .1

Next, I build the transition matrix using a 5-gram approximation to English.  That is, I have a dictionary in which keys are 5-grams and values are their empirical probability, given by the wikipedia corpus

In [21]:
def build_transition_matrix():
	
	print('building matrix...')
	#preallocate ngram list
	ngram_list = np.empty(corpus_sz,dtype=object)

	#get all the ngrams that are in the mined text
	for i in range(corpus_sz):
		ngram_list[i] = cleaned_corpus[i:i+n]

	corpus_ngrams = set(ngram_list)
	ngram2freq = Counter(ngram_list)
	total_ngrams = sum(ngram2freq.values())

	#transition matrix Q as dictionary: Q[ngram] = p_hat(ngram)
	transition_matrix = {}
	
	for perm in enumerate(all_possible_ngrams):
		ngram = ''.join(list(perm[1]))
		
		# if the ngram is not absolute garbage, store it in Q
		if ngram in corpus_ngrams:
			# empirical probabiliy
			transition_matrix[ngram] = (1/total_ngrams)*(ngram2freq.get(ngram,min_value))

	print('done!')
	return transition_matrix 

Below will build a transistion matrix for you if you don't already have it

In [25]:
if os.path.exists('transition_matrix.pickle'):
		transition_matrix = pickle.load(open('transition_matrix.pickle','rb'))
else:
		transition_matrix = build_transition_matrix()
		pickle.dump(transition_matrix, open('transition_matrix.pickle','wb'))

Now, let's make some helper functions for MCMC

In [26]:
# this is the first step: picking a starting point
def select_sigma(alphabet):
	# picks a random permutation of the alphabet
	alphabet = list(alphabet)
	random.shuffle(alphabet)
	return ''.join(alphabet)

# encrypts a message using the cipher 
def encode(plaintext, sigma, alphabet):
	# sigma: {a,b,c,...} -> {hfaiduzhi}
	sigmaMap = dict(zip(alphabet, sigma))
	return ''.join(sigmaMap.get(char.lower(), char) for char in plaintext)

# decrypts the message
def decode(ciphertext, sigma, alphabet):
	# sigma^-1: {hfaiduzhi} -> {a,b,c,...}
	sigmaMap = dict(zip(sigma, alphabet))
	return ''.join(sigmaMap.get(char.lower(), char) for char in ciphertext)

#this is how we move from one alphabet to the next
def swap_letters(alphabet):
	#choose two random spots in the alphabet
	chosen_first_place = random.randint(0,alphabet_sz-1)
	chosen_second_place = random.randint(0,alphabet_sz-1)
	#make sure they're not equal
	while chosen_second_place == chosen_first_place:
		chosen_second_place = random.randint(0,alphabet_sz-1)
	#because strings are immutable, need to make a list before accessing
	alphabet = list(alphabet)
	#swap
	first = alphabet[chosen_first_place]
	second = alphabet[chosen_second_place]
	alphabet[chosen_second_place] = first
	alphabet[chosen_first_place] = second
	#return new alphabet as a string
	alphabet = ''.join(alphabet)
	return alphabet


The basic idea is stochastic optimization.  We start out with a random permutation of the alphabet, and hopefully by swapping letters in that alphabet, we obtain the original substitution cipher.  The way in which we decide to swap or not corresponds to minimizing this energy function.  In this way, we descend down the energy landscape until we reach some sort of minimum.  First, let's look at the energy calculation...

In [24]:
def get_energy(decoded):

	#preallocate
	present_ngrams = np.empty(len(decoded),dtype=object)

	#make all the ngrams
	for i in range(len(decoded)):
		present_ngrams[i] = decoded[i:i+n]

	# using the formula for energy
	return -np.sum(np.log([transition_matrix.get(ngram,min_value) for ngram in present_ngrams]))

Next, the actual Metropolis algorithm which seeks to minimize energy 

In [27]:
def run_metropolis(ciphertext,max_iterations):	
	
	#initial guess
	sigma = select_sigma(alphabet_string)
	decoded = decode(ciphertext,sigma,alphabet_string)
	energy_sigma = get_energy(decoded)
	delta_energy = energy_sigma

	curr_text = ciphertext
	iters = 0

	while iters<=max_iterations:

		if iters%(max_iterations/10) == 0:
			print('===============================================================')
			print('iteration: ', iters)
			print('result: \n', decoded)

		tau = swap_letters(sigma)
		decoded = decode(curr_text,tau,alphabet_string)
		energy_tau = get_energy(decoded)
		delta_energy = energy_tau - energy_sigma
		
		if delta_energy < 0:
			#accept
			sigma = tau
			energy_sigma = energy_tau
			# if applying sigma to the current text retrieves the original ciphertext
			if encode(curr_text, sigma, alphabet_string) == ciphertext:
				# we're done
				break;
		else:
			#reject: accept with prob exp(-beta*delta_V)
			p = np.exp(-beta*delta_energy)
			U = np.random.uniform(0,1)
			if U <= p:
				sigma = tau
				energy_sigma = energy_tau

		iters = iters+1

	print('========================= DONE! ================================')
	return decoded

Time to run it on the student text!

In [29]:
student_text_path = 'scrambled/m_student.txt'
with open(student_text_path) as f:
	student = f.read()
	#some preprocessing 
	student = student.split(':')[1]
	student = clean_text(student)

	#found this by experimenting
	max_iterations = 10000
	output = run_metropolis(student,max_iterations)

iteration:  0
result: 
 bfmqbwahiqfbyaubbsbefzllqxbabemshfbvhbftvbsjfvbc bvoxbwahiqfbyaubfzwnqxbsfbzjxqhbc bahcbajxbefahfqxblvhbwaiqbmvhjbajxbfmqbiawslswbdzsffsjubfmqbuvvxbwsf bvlbvoxbcajmaffvbsbxzo bahhsgqxbsjbjqtbyqxlvhxbsfbtaebabeafzhxa bjsumfbsjbxqwqcyqhbczwmbtaebsbxseaiivsjfqxbzivjboqahjsjubfmafbfmqbosffoqbiawnqfblvhbjajfzwnqfbmaxbaohqax beasoqxbajxbfmafbjvbta bvlbhqawmsjubfmafbioawqbtvzoxbvllqhbfsobfmqblvovtsjubcvjxa bbaebcvefb vzjubwajxsxafqeblvhbfmqbiasjebajxbiqjaofsqebvlbtmaosjubefvibafbfmsebeacqbjqtbyqxlvhxbfmqjwqbfvbqcyahnbvjbfmqshbgv auqbsfbca baebtqobyqbhqoafqxbfmafbsblvhbvjqbmaxbjvbsxqabvlbevbxvsjublvhbc bcsjxbtaebcaxqbzibfvbeasobsjbjvbvfmqhbfmajbabjajfzwnqfbwhalfbyqwazeqbfmqhqbtaebablsjqbyvsefqhvzebevcqfmsjubayvzfbqgqh fmsjubwvjjqwfqxbtsfmbfmafblacvzebvoxbseoajxbtmswmbacapsjuo bioqaeqxbcqbyqesxqebfmvzumbjqtbyqxlvhxbmaebvlboafqbyqqjbuhaxzao bcvjvivosesjubfmqbyzesjqeebvlbtmaosjubajxbfmvzumbsjbfmsebcaffqhbivvhbvoxbjajfzwnqfbsebjvtbczwmbyqmsjxbmqhb qfbjajfzwnqfbtaebmqhbuhqaf

iteration:  2000
result: 
  the carpet bag  i stuffed a shirt or two into xy old carpet bag tucked it under xy arx and started for cape horn and the pacific quitting the good city of old xanhatto i duly arrived in new bedford it was a saturday night in decexber xuch was i disappointed upon learning that the little packet for nantucket had already sailed and that no way of reaching that place would offer til the folowing xonday  as xost young candidates for the pains and penalties of whaling stop at this saxe new bedford thence to exbark on their voyage it xay as wel be related that i for one had no idea of so doing for xy xind was xade up to sail in no other than a nantucket craft because there was a fine boisterous soxething about everything connected with that faxous old island which axazingly pleased xe besides though new bedford has of late been gradualy xonopolising the business of whaling and though in this xatter poor old nantucket is now xuch behind her yet nantucket was her gr

iteration:  4000
result: 
  tie carpet bag  h stuffed a sihrt or two hnto my old carpet bag tucked ht under my arm and started for cape iorn and tie pachfhc quhtthng tie good chty of old maniatto h duly arrhved hn new bedford ht was a saturday nhgit hn december muci was h dhsappohnted upon learnhng tiat tie lhttle packet for nantucket iad already sahled and tiat no way of reacihng tiat place would offer thl tie folowhng monday  as most young candhdates for tie pahns and penalthes of wialhng stop at tihs same new bedford tience to embark on tiehr voyage ht may as wel be related tiat h for one iad no hdea of so dohng for my mhnd was made up to sahl hn no otier tian a nantucket craft because tiere was a fhne bohsterous sometihng about everytihng connected whti tiat famous old hsland wihci amazhngly pleased me beshdes tiougi new bedford ias of late been gradualy monopolhshng tie bushness of wialhng and tiougi hn tihs matter poor old nantucket hs now muci beihnd ier yet nantucket was ier gr

iteration:  6000
result: 
  the carpet bag  i stuffed a shirt or two ivto my old carpet bag tucked it uvder my arm avd started for cape horv avd the pacific quittivg the good city of old mavhatto i duly arrined iv vew bedford it was a saturday vight iv december much was i disappoivted upov learvivg that the little packet for vavtucket had already sailed avd that vo way of reachivg that place would offer til the folowivg movday  as most youvg cavdidates for the paivs avd pevalties of whalivg stop at this same vew bedford thevce to embark ov their noyage it may as wel be related that i for ove had vo idea of so doivg for my mivd was made up to sail iv vo other thav a vavtucket craft because there was a five boisterous somethivg about enerythivg covvected with that famous old islavd which amazivgly pleased me besides though vew bedford has of late beev gradualy movopolisivg the busivess of whalivg avd though iv this matter poor old vavtucket is vow much behivd her yet vavtucket was her gr

iteration:  8000
result: 
  the cbrpet abg  i stuffed b shirt or two into my old cbrpet abg tucked it under my brm bnd stbrted for cbpe horn bnd the pbcific quitting the good city of old mbnhbtto i duly brrived in new aedford it wbs b sbturdby night in decemaer much wbs i disbppointed upon lebrning thbt the little pbcket for nbntucket hbd blrebdy sbiled bnd thbt no wby of rebching thbt plbce would offer til the folowing mondby  bs most young cbndidbtes for the pbins bnd penblties of whbling stop bt this sbme new aedford thence to emabrk on their voybge it mby bs wel ae relbted thbt i for one hbd no ideb of so doing for my mind wbs mbde up to sbil in no other thbn b nbntucket crbft aecbuse there wbs b fine aoisterous something baout everything connected with thbt fbmous old islbnd which bmbzingly plebsed me aesides though new aedford hbs of lbte aeen grbdubly monopolising the ausiness of whbling bnd though in this mbtter poor old nbntucket is now much aehind her yet nbntucket wbs her gr

iteration:  10000
result: 
  the carpet bag  i stuffed a shirt or two ixto my old carpet bag tucked it uxder my arm axd started for cape horx axd the pacific quittixg the good city of old maxhatto i duly arrived ix xew bedford it was a saturday xight ix december much was i disappoixted upox learxixg that the little packet for xaxtucket had already sailed axd that xo way of reachixg that place would offer til the folowixg moxday  as most youxg caxdidates for the paixs axd pexalties of whalixg stop at this same xew bedford thexce to embark ox their voyage it may as wel be related that i for oxe had xo idea of so doixg for my mixd was made up to sail ix xo other thax a xaxtucket craft because there was a fixe boisterous somethixg about everythixg coxxected with that famous old islaxd which amazixgly pleased me besides though xew bedford has of late beex gradualy moxopolisixg the busixess of whalixg axd though ix this matter poor old xaxtucket is xow much behixd her yet xaxtucket was her g

Although the text is not English, it's close enough to readable, hence, decoded.  It reads,

*"the carpet bag  i stuffed a shirt or two ixto my old carpet bag tucked it uxder my arm axd started for cape horx axd the pacific quittixg the good city of old maxhatto i duly arrived ix xew bedford it was a saturday xight ix december much was i disappoixted upox learxixg that the little packet for xaxtucket had already sailed axd that xo way of reachixg that place would offer til the folowixg moxday  as most youxg caxdidates for the paixs axd pexalties of whalixg stop at this same xew bedford thexce to embark ox their voyage it may as wel be related that i for oxe had xo idea of so doixg for my mixd was made up to sail ix xo other thax a xaxtucket craft because there was a fixe boisterous somethixg about everythixg coxxected with that famous old islaxd which amazixgly pleased me besides though xew bedford has of late beex gradualy moxopolisixg the busixess of whalixg axd though ix this matter poor old xaxtucket is xow much behixd her yet xaxtucket was her great origixal the tyre of this carthage the place where the first dead americax whale was straxded where else but from xaxtucket did those aborigixal whalemex the red mex first saly out ix caxoes to give chase to the leviathax axd where but from xaxtucket too did that first advexturous little sloop put forth partly ladex with imported cobblestoxes so goes the story to throw at the whales ix order to discover whex they were xigh exough to risk a harpoox from the bowsprit  xow havixg a xight a day axd stil axother xight folowixg before me ix xew bedford ere i could embark for my destixed port it became a matter of coxcerxmext where i was to eat axd sleep meaxwhile it was a very dubious lookixg xay a very dark axd dismal xight bitixgly cold axd cheerless i kxew xo oxe ix the place with axnious grapxels i had souxded my pocket axd oxly brought up a few pieces of silver so wherever you go ishmael said i to myself as i stood ix the middle of a dreary street shoulderixg my bag axd comparixg the gloom towards the xorth with the darkxess towards the south wherever ix your wisdom you may coxclude to lodge for the xight my dear ishmael be sure to ixquire the price axd doxt be too particular  with haltixg steps i paced the streets axd passed the sigx of the crossed harpooxs but it looked too enpexsive axd joly there further ox from the bright red wixdows of the sword fish ixx there came such fervext rays that it seemed to have melted the packed sxow axd ice from before the house for everywhere else the coxgealed frost lay tex ixches thick ix a hard asphaltic pavemext rather weary for me whex i struck my foot agaixst the flixty projectioxs because from hard remorseless service the soles of my boots were ix a most miserable plight too enpexsive axd joly agaix thought i pausixg oxe momext to watch the broad glare ix the street axd hear the souxds of the tixklixg glasses withix but go ox ishmael said i at last doxt you hear get away from before the door your patched boots are stoppixg the way so ox i wext i xow by ixstixct folowed the streets that took me waterward for there doubtless were the cheapest if xot the cheeriest ixxs  such dreary streets blocks of blackxess xot houses ox either haxd axd here axd there a caxdle like a caxdle movixg about ix a tomb at this hour of the xight of the last day of the week that quarter of the towx proved al but deserted but presextly i came to a smoky light proceedixg from a low wide buildixg the door of which stood ixvitixgly opex it had a careless look as if it were meaxt for the uses of the public so exterixg the first thixg i did was to stumble over ax ash bon ix the porch ha thought i ha as the flyixg particles almost choked me are these ashes from that destroyed city gomorrah but the crossed harpooxs axd the sword fish this thex must xeeds be the sigx of the trap however i picked myself up axd hearixg a loud voice withix pushed ox axd opexed a secoxd ixterior door  it seemed the great black parliamext sittixg ix tophet a huxdred black faces turxed rouxd ix their rows to peer axd beyoxd a black axgel of doom was beatixg a book ix a pulpit it was a xegro church axd the preachers tent was about the blackxess of darkxess axd the weepixg axd wailixg axd teeth gxashixg there ha ishmael muttered i backixg out wretched extertaixmext at the sigx of the trap  movixg ox i at last came to a dim sort of light xot far from the docks axd heard a forlorx creakixg ix the air axd lookixg up saw a swixgixg sigx over the door with a white paixtixg upox it faixtly represextixg a tal straight jet of misty spray axd these words uxderxeath the spouter ixx peter coffix  coffix spouter rather omixous ix that particular coxxeniox thought i but it is a commox xame ix xaxtucket they say axd i suppose this peter here is ax emigraxt from there as the light looked so dim axd the place for the time looked quiet exough axd the dilapidated little woodex house itself looked as if it might have beex carted here from the ruixs of some burxt district axd as the swixgixg sigx had a poverty strickex sort of creak to it i thought that here was the very spot for cheap lodgixgs axd the best of pea coffee  it was a queer sort of place a gable exded old house oxe side palsied as it were axd leaxixg over sadly it stood ox a sharp bleak corxer where that tempestuous wixd euroclydox kept up a worse howlixg thax ever it did about poor pauls tossed craft euroclydox xevertheless is a mighty pleasaxt zephyr to axy oxe ix doors with his feet ox the hob quietly toastixg for bed ix judgixg of that tempestuous wixd caled euroclydox says ax old writer of whose works i possess the oxly copy entaxt it maketh a marvelous differexce whether thou lookest out at it from a glass wixdow where the frost is al ox the outside or whether thou observest it from that sashless wixdow where the frost is ox both sides axd of which the wight death is the oxly glazier true exough thought i as this passage occurred to my mixd old black letter thou reasoxest wel yes these eyes are wixdows axd this body of mixe is the house what a pity they didxt stop up the chixks axd the craxxies though axd thrust ix a little lixt here axd there but its too late to make axy improvemexts xow the uxiverse is fixished the copestoxe is ox axd the chips were carted off a miliox years ago poor lazarus there chatterixg his teeth agaixst the curbstoxe for his pilow axd shakixg off his tatters with his shiverixgs he might plug up both ears with rags axd put a corx cob ixto his mouth axd yet that would xot keep out the tempestuous euroclydox euroclydox says old dives ix his red silkex wrapper he had a redder oxe afterwards pooh pooh what a fixe frosty xight how oriox glitters what xortherx lights let them talk of their oriextal summer climes of everlastixg coxservatories give me the privilege of makixg my owx summer with my owx coals  but what thixks lazarus cax he warm his blue haxds by holdixg them up to the graxd xortherx lights would xot lazarus rather be ix sumatra thax here would he xot far rather lay him dowx lexgthwise aloxg the lixe of the equator yea ye gods go dowx to the fiery pit itself ix order to keep out this frost  xow that lazarus should lie straxded there ox the curbstoxe before the door of dives this is more woxderful thax that ax iceberg should be moored to oxe of the moluccas yet dives himself he too lives like a czar ix ax ice palace made of frozex sighs axd beixg a presidext of a temperaxce society he oxly drixks the tepid tears of orphaxs  but xo more of this blubberixg xow we are goixg a whalixg axd there is plexty of that yet to come let us scrape the ice from our frosted feet axd see what sort of a place this spouter may be".  *

Clearly, we would retrieve the actual message by swapping 'n' and 'x'


The next passage was more challenging.

In [30]:
challenge_1_path = 'scrambled/rrr_student.txt'
with open(challenge_1_path) as f:
	challenge_1 = f.read()
	challenge_1 = challenge_1.split(':')[1]
	challenge_1 = clean_text(challenge_1)

	#found this by experimenting
	max_iterations = 100000
	output = run_metropolis(challenge_1,max_iterations)

iteration:  0
result: 
 vxlxoyfu ovvxkoqsxcvrkg vqlbb x ivkxvoq vblnnlc vtkgoqwvoq vr xoskxsxcvnkvlmo gvlvmsx vgyxvt vnlm pwvlggsa ivsxvxlxoyfu ovvxlxoyfu ovolu vkyovwkygvrlbvlxivpkkuvlovsovn  vtqlovlvg lpvfkgx gvkmvoq vtkgpivsovkffybs nvqktvsovnolxinvoq g vltlwvkmmvnqkg vrkg vpkx pwvoqlxvoq v iiwnokx vpscqoqkyn vpkkuvlovsovlvr g vqsppkfuvlxiv pdktvkmvnlxivlppvd lfqvtsoqkyovlvdlfucgkyxivoq g vsnvrkg vnlxivoq g voqlxvwkyvtkypivyn vsxvot xowvw lgnvlnvlvnydnosoyo vmkgvdpkosxcvblb gvnkr vclr nkr vtscqonvtsppvo ppvwkyvoqlovoq wvqla vokvbplxovt  invoq g voq wvikxovcgktvxloyglppwvoqlovoq wvsrbkgovflxlilvoqsnop nvoqlovoq wvqla vokvn xivd wkxivn lnvmkgvlvnbsp vokvnokbvlvp luvsxvlxvkspvflnuvoqlovbs f nvkmvtkkivsxvxlxoyfu ovlg vflggs ivldkyovpsu vdsonvkmvoq vogy vfgknnvsxvgkr voqlovb kbp voq g vbplxovoklinokkpnvd mkg voq sgvqkyn nvokvc ovyxi gvoq vnqli vsxvnyrr gvosr voqlovkx vdpli vkmvcglnnvrlu nvlxvklnsnvoqg  vdpli nvsxvlvilwnvtlpuvlvbglsgs voqlovoq wvt lgvjysfunlxivnqk nvnkr oqsxcvpsu vplbplxi

iteration:  20000
result: 
  nantwcket  nothing more happened on the passage uorthy the mentioning so after a fine rwn ue safely arrived in nantwcket  nantwcket take owt yowr map and look at it see uhat a real corner of the uorld it occwpies hou it stands there auay off shore more lonely than the eddystone lighthowse look at it a mere hillock and elbou of sand all beach uithowt a backgrownd there is more sand there than yow uowld wse in tuenty years as a swbstitwte for bloting paper some gamesome uights uill tell yow that they have to plant ueeds there they dont grou natwrally that they import canada thistles that they have to send beyond seas for a spile to stop a leak in an oil cask that pieces of uood in nantwcket are carried abowt like bits of the trwe cross in rome that people there plant toadstools before their howses to get wnder the shade in swmmer time that one blade of grass makes an oasis three blades in a days ualk a prairie that they uear qwicksand shoes something like lap

iteration:  40000
result: 
 pnantucketppnothingpmorepha  enedponpthep assagepworthypthepmentioningpsopafterpapfineprunpwepsafelyparrivedpinpnantucketppnantucketptakepoutpyourpma pandplookpatpitpseepwhatpaprealpcornerpofpthepworldpitpoccu iesphowpitpstandsptherepawaypoffpshorepmoreplonelypthanpthepeddystoneplighthouseplookpatpitpapmerephillockpandpelbowpofpsandpallpbeachpwithoutpapbackgroundptherepispmorepsandptherepthanpyoupwouldpusepinptwentypyearspaspapsubstitutepforpblotingp a erpsomepgamesomepwightspwillptellpyoupthatptheyphaveptop lantpweedspthereptheypdontpgrowpnaturallypthatptheypim ortpcanadapthistlespthatptheyphaveptopsendpbeyondpseaspforpaps ileptopsto papleakpinpanpoilpcaskpthatp iecespofpwoodpinpnantucketparepcarriedpaboutplikepbitspofptheptruepcrosspinpromepthatp eo leptherep lantptoadstoolspbeforeptheirphousesptopgetpunderpthepshadepinpsummerptimepthatponepbladepofpgrasspmakespanpoasispthreepbladespinpapdayspwalkpap rairiepthatptheypwearpquicksandpshoespsomethingplikepla 

iteration:  60000
result: 
  nantucket  nothing more happened on the passage worthy the mentioning so after a fine run we safely arrived in nantucket  nantucket take out your map and look at it see what a real corner of the world it occupies how it stands there away off shore more lonely than the eddystone lighthouse look at it a mere hillock and elzow of sand all zeach without a zackground there is more sand there than you would use in twenty years as a suzstitute for zloting paper some gamesome wights will tell you that they have to plant weeds there they dont grow naturally that they import canada thistles that they have to send zeyond seas for a spile to stop a leak in an oil cask that pieces of wood in nantucket are carried azout like zits of the true cross in rome that people there plant toadstools zefore their houses to get under the shade in summer time that one zlade of grass makes an oasis three zlades in a days walk a prairie that they wear quicksand shoes something like lap

iteration:  80000
result: 
  nantucket  nothing more happened on the passage worthy the mentioning so alter a line run we salefy arrived in nantucket  nantucket take out your map and fook at it see what a reaf corner ol the worfd it occupies how it stands there away oll shore more fonefy than the eddystone fighthouse fook at it a mere hiffock and efbow ol sand aff beach without a background there is more sand there than you woufd use in twenty years as a substitute lor bfoting paper some gamesome wights wiff teff you that they have to pfant weeds there they dont grow naturaffy that they import canada thistfes that they have to send beyond seas lor a spife to stop a feak in an oif cask that pieces ol wood in nantucket are carried about fike bits ol the true cross in rome that peopfe there pfant toadstoofs belore their houses to get under the shade in summer time that one bfade ol grass makes an oasis three bfades in a days wafk a prairie that they wear quicksand shoes something fike fap

iteration:  100000
result: 
  cactunket  cothicg more happeced oc the passage worthy the mectiocicg so after a fice ruc we safely arrived ic cactunket  cactunket take out your map acd look at it see what a real norcer of the world it onnupies how it stacds there away off shore more locely thac the eddystoce lighthouse look at it a mere hillonk acd elbow of sacd all beanh without a bankgroucd there is more sacd there thac you would use ic twecty years as a substitute for bloticg paper some gamesome wights will tell you that they have to plact weeds there they doct grow caturally that they import nacada thistles that they have to secd beyocd seas for a spile to stop a leak ic ac oil nask that pienes of wood ic cactunket are narried about like bits of the true nross ic rome that people there plact toadstools before their houses to get ucder the shade ic summer time that oce blade of grass makes ac oasis three blades ic a days walk a prairie that they wear quinksacd shoes somethicg like la

This passage reads,

cactunket  cothicg more happeced oc the passage worthy the mectiocicg so after a fice ruc we safely arrived ic cactunket  cactunket take out your map acd look at it see what a real norcer of the world it onnupies how it stacds there away off shore more locely thac the eddystoce lighthouse look at it a mere hillonk acd elbow of sacd all beanh without a bankgroucd there is more sacd there thac you would use ic twecty years as a substitute for bloticg paper some gamesome wights will tell you that they have to plact weeds there they doct grow caturally that they import nacada thistles that they have to secd beyocd seas for a spile to stop a leak ic ac oil nask that pienes of wood ic cactunket are narried about like bits of the true nross ic rome that people there plact toadstools before their houses to get ucder the shade ic summer time that oce blade of grass makes ac oasis three blades ic a days walk a prairie that they wear quinksacd shoes somethicg like laplacder scow shoes that they are so shut up belted about every way icnlosed surroucded acd made ac uter islacd of by the oneac that to their very nhairs acd tables small nlams will sometimes be foucd adhericg as to the banks of sea turtles but these extravagaczas ocly show that cactunket is co illicois  look cow at the wocdrous traditiocal story of how this islacd was setled by the red mec thus goes the legecd ic oldec times ac eagle swooped dowc upoc the cew ecglacd noast acd narried off ac icfact icdiac ic his talocs with loud lamect the parects saw their nhild borce out of sight over the wide waters they resolved to follow ic the same direntioc seticg out ic their nacoes after a perilous passage they disnovered the islacd acd there they foucd ac empty ivory nasket the poor litle icdiacs skeletoc  what wocder thec that these cactunketers borc oc a beanh should take to the sea for a livelihood they first naught nrabs acd quohogs ic the sacd growc bolder they waded out with cets for mankerel more experiecned they pushed off ic boats acd naptured nod acd at last laucnhicg a cavy of great ships oc the sea explored this watery world put ac icnessact belt of nirnumcavigatiocs roucd it peeped ic at behricgs straits acd ic all seasocs acd all oneacs denlared everlasticg war with the mightiest acimated mass that has survived the flood most mocstrous acd most mouctaicous that himmalehac salt sea mastodoc nlothed with sunh portectouscess of ucnocsnious power that his very pacins are more to be dreaded thac his most fearless acd malinious assaults  acd thus have these caked cactunketers these sea hermits issuicg from their act hill ic the sea overruc acd nocquered the watery world like so macy alexacders parnellicg out amocg them the atlactin panifin acd icdiac oneacs as the three pirate powers did polacd let amerina add mexino to texas acd pile nuba upoc nacada let the ecglish overswarm all icdia acd hacg out their blazicg baccer from the suc two thirds of this terraqueous globe are the cactunketers for the sea is his he owcs it as emperors owc empires other seamec havicg but a right of way through it mernhact ships are but extecsioc bridges armed oces but floaticg forts evec pirates acd privateers though followicg the sea as highwaymec the road they but plucder other ships other fragmects of the lacd like themselves without seekicg to draw their livicg from the botomless deep itself the cactunketer he aloce resides acd riots oc the sea he aloce ic bible lacguage goes dowc to it ic ships to acd fro ploughicg it as his owc spenial plactatioc there is his home there lies his busicess whinh a coahs flood would cot icterrupt though it overwhelmed all the milliocs ic nhica he lives oc the sea as prairie nonks ic the prairie he hides amocg the waves he nlimbs them as nhamois hucters nlimb the alps for years he kcows cot the lacd so that whec he nomes to it at last it smells like acother world more stracgely thac the mooc would to ac earthsmac with the lacdless gull that at sucset folds her wicgs acd is ronked to sleep betweec billows so at cightfall the cactunketer out of sight of lacd furls his sails acd lays him to his rest while ucder his very pillow rush herds of walruses acd whales"

To retrieve the original text, swap 'n' and 'c'

Finally, the second challenge text.

In [ ]:
challenge_2_path = 'scrambled/zzz_student.txt'
with open(challenge_2_path) as f:
	challenge_2 = f.read()
	challenge_2 = challenge_2.split(':')[1]
	challenge_2 = clean_text(challenge_2)

	#found this by experimenting
	max_iterations = 1000000
	output = run_metropolis(challenge_2,max_iterations)

iteration:  0
result: 
 ijumxuhfzidklszizftxm aidxmikuhfzxuxhiizf ijduxizlwibdukirdamizdbfz hliwdbbuxhix cikdbfuxhbidaltxmiuziitjitjitjibzlwizfdzizftxm aiwk xzrizllijtpfizftxm aitwif a icfdzbizf itb ilsizftxm aitjitjitjic imlxzicdxzizftxm aic icdxziatjihue itbidihkdbbilsiatjitjitjitjiii
iteration:  100000
result: 
 ahpmopleratuscraredomxbatomaupleropolaarexahtpoarskaitpuagtbmartierxlsaktiipolaoxnautiepoliatbsdomapraadhadhadhairskaretraredomxbakuxorgarssahdfearedomxbadkaexbxanetriarexadixascaredomxbadhadhadhanxamsorantoraredomxbanxantorabdhalpyxadiatalutiiascabdhadhadhadhaaa
iteration:  200000
result: 
 ahcmoclwratusprarwdom batomauclwrocolaarw ahtcoarskaitcuagtbmartiwr lsaktiicolao nautiwcoliatbsdomacraadhadhadhairskarwtrarwdom baku orgarssahdfwarwdom badkaw b anwtriarw adi asparwdom badhadhadhan amsorantorarwdom ban antorabdhalcy adiatalutiiaspabdhadhadhadhaaa
iteration:  300000
result: 
 ahpmopleratuscraredom yatomaupleropolaare ahtpoarskaitpuagtymartier lsaktiipolao nautiepoliatysdo

I wasn't able to decode this passage.  I know that it contains dialogue of two drunk people talking, so it probably contains very unlikely combinations of letters for typical English.  Since my corpus was Wikipedia, it is doubtful that my transistion matrix contains the info I would need to decode this passage.